In [ ]:
# Odoo connection details
ODOO_URL = URL = "https://kargo2.odoo.com"  # e.g., https://kargofit.odoo.com
ODOO_DB = DB = "kargo2"  # e.g., kargofit
ODOO_USERNAME = USERNAME = "admin@kargofit.com"
ODOO_PASSWORD = PASSWORD = "af74742249c06aa4aee821b557c33f0d54267971"

In [ ]:
import xmlrpc.client
import json

# Authenticate
common = xmlrpc.client.ServerProxy(f"{URL}/xmlrpc/2/common")
uid = common.authenticate(DB, USERNAME, PASSWORD, {})

# Connect to the object endpoint
models = xmlrpc.client.ServerProxy(f"{URL}/xmlrpc/2/object")


def fetch_products(uid, domain=None, fields=None, limit=100, offset=0):
    """
    Fetch products from Odoo
    
    Args:
        uid: User ID from authentication
        domain: Search domain (list of tuples), e.g., [('type', '=', 'product')]
        fields: List of fields to retrieve, e.g., ['name', 'list_price', 'qty_available']
        limit: Maximum number of records to fetch
        offset: Number of records to skip
    
    Returns:
        List of product dictionaries
    """
    models = xmlrpc.client.ServerProxy(f'{ODOO_URL}/xmlrpc/2/object')
    
    # Default domain (empty = all products)
    if domain is None:
        domain = []
    
    # Default fields (common product fields)
    if fields is None:
        fields = [
            'id',
            'name',
            'default_code',  # Internal reference/SKU
            'barcode',
            'list_price',    # Sale price
            'standard_price', # Cost price
            'type',          # Product type (product, service, consu)
            'categ_id',      # Product category
            'uom_id',        # Unit of measure
            'active',
            'description',
            'description_sale'
        ]
    
    # Search and read products
    product_ids = models.execute_kw(
        ODOO_DB, uid, ODOO_PASSWORD,
        'product.product', 'search',
        [domain],
        {'limit': limit, 'offset': offset}
    )
    
    if not product_ids:
        print("No products found.")
        return []
    
    products = models.execute_kw(
        ODOO_DB, uid, ODOO_PASSWORD,
        'product.product', 'read',
        [product_ids],
        {'fields': fields}
    )
    
    return products

def fetch_product_templates(uid, domain=None, fields=None, limit=100):
    """
    Fetch product templates from Odoo (product.template)
    Product templates are the generic products, while product.product includes variants
    """
    models = xmlrpc.client.ServerProxy(f'{ODOO_URL}/xmlrpc/2/object')
    
    if domain is None:
        domain = []
    
    if fields is None:
        fields = [
            'id',
            'name',
            'list_price',
            'type',
            'categ_id',
            'active'
        ]
    
    template_ids = models.execute_kw(
        ODOO_DB, uid, ODOO_PASSWORD,
        'product.template', 'search',
        [domain],
        {'limit': limit}
    )
    
    if not template_ids:
        print("No product templates found.")
        return []
    
    templates = models.execute_kw(
        ODOO_DB, uid, ODOO_PASSWORD,
        'product.template', 'read',
        [template_ids],
        {'fields': fields}
    )
    
    return templates

def main():
    """Main function to demonstrate fetching product data"""
    try:
        
        # Example 1: Fetch all products with default fields
        print("\n=== Fetching Products ===")
        products = fetch_products(uid, limit=10)
        print(f"Found {len(products)} products")
        
        # Display first product as example
        if products:
            print("\nFirst product:")
            print(json.dumps(products[0], indent=2, default=str))
        
        # Example 2: Fetch only active products with specific fields
        print("\n=== Fetching Active Products with Specific Fields ===")
        active_products = fetch_products(
            uid,
            domain=[('active', '=', True)],
            fields=['name', 'list_price'],
            limit=5
        )
        
        for product in active_products:
            print(f"- {product['name']}: ${product.get('list_price', 0)} ")
        
        # Example 3: Fetch products by type (e.g., storable products only)
        print("\n=== Fetching Storable Products ===")
        storable_products = fetch_products(
            uid,
            domain=[('type', '=', 'product')],
            limit=5
        )
        print(f"Found {len(storable_products)} storable products")
        
        # Example 4: Fetch product templates
        print("\n=== Fetching Product Templates ===")
        templates = fetch_product_templates(uid, limit=5)
        print(f"Found {len(templates)} product templates")
        
        if templates:
            print("\nFirst template:")
            print(json.dumps(templates[0], indent=2, default=str))
        
    except Exception as e:
        print(f"Error: {str(e)}")

if __name__ == "__main__":
    main()

In [ ]:
"""
Script to fetch all sale orders created today with current status from Odoo using XML-RPC
"""

from datetime import datetime, date, timedelta
import json
import xmlrpc.client
import os


def fetch_from_odoo():
    """Fetch sales orders from Odoo"""
    
    try:
        # Authenticate
        common = xmlrpc.client.ServerProxy(f'{ODOO_URL}/xmlrpc/2/common')
        uid = common.authenticate(ODOO_DB, USERNAME, ODOO_PASSWORD, {})
        
        if not uid:
            print("❌ Authentication failed. Please check your credentials.")
            return []
        
        print("✓ Authentication successful")
        
        # Connect to object endpoint
        models = xmlrpc.client.ServerProxy(f'{ODOO_URL}/xmlrpc/2/object')
        
        # Get today's date range
        today = date.today().strftime('%Y-%m-%d')
        tomorrow = (date.today() + timedelta(days=1)).strftime('%Y-%m-%d')
        
        print(f"✓ Searching for orders created on: {today}")
        
        # Search for today's sale orders
        order_ids = models.execute_kw(
            ODOO_DB, uid, ODOO_PASSWORD,
            'sale.order', 'search',
            [[['create_date', '>=', f'{today} 00:00:00'], 
              ['create_date', '<', f'{tomorrow} 00:00:00']]]
        )
        
        print(f"✓ Found {len(order_ids)} order(s)")
        
        # Read order details
        if order_ids:
            orders = models.execute_kw(
                ODOO_DB, uid, ODOO_PASSWORD,
                'sale.order', 'read',
                [order_ids],
                {'fields': ['name', 'partner_id', 'date_order', 'state', 
                           'amount_total', 'amount_untaxed', 'amount_tax', 
                           'create_date', 'user_id', 'company_id']}
            )
            
            # Extract partner IDs to fetch customer details
            partner_ids = [order['partner_id'][0] for order in orders if order['partner_id']]
            
            # Fetch customer details including reference/customer number and phone
            customers = {}
            if partner_ids:
                customer_data = models.execute_kw(
                    ODOO_DB, uid, ODOO_PASSWORD,
                    'res.partner', 'read',
                    [partner_ids],
                    {'fields': ['id', 'name', 'ref', 'phone_sanitized']}  # Added phone and mobile fields
                )
                customers = {c['id']: c for c in customer_data}
            
            # Format results
            results = []
            for order in orders:
                partner_id = order['partner_id'][0] if order['partner_id'] else None
                customer = customers.get(partner_id, {})
                
                # Get phone number (prefer mobile, fallback to phone)
                customer_phone = customer.get('phone_sanitized') or 'N/A'
                
                results.append({
                    'order_id': order['id'],
                    'order_number': order['name'],
                    'customer_id': partner_id or 'N/A',
                    'customer_number': customer.get('ref', 'N/A'),
                    'customer_name': order['partner_id'][1] if order['partner_id'] else 'N/A',
                    'customer_phone': customer_phone,
                    'order_date': order['date_order'],
                    'status': order['state'],
                    'total_amount': order['amount_total'],
                    'untaxed_amount': order.get('amount_untaxed', 0),
                    'tax_amount': order.get('amount_tax', 0),
                    'salesperson': order['user_id'][1] if order.get('user_id') else 'N/A',
                    'company': order['company_id'][1] if order.get('company_id') else 'N/A',
                    'created_at': order['create_date']
                })
            return results
        
        return []
    
    except Exception as e:
        print(f"❌ Error connecting to Odoo: {e}")
        return []



def display_orders(orders):
    """Display orders in a formatted way"""
    
    print("=" * 80)
    print(f"Sales Orders Created Today: {date.today().strftime('%Y-%m-%d')}")
    print("=" * 80)
    
    if not orders:
        print("\n📭 No sales orders found for today.")
        return
    
    print(f"\n📊 Total Orders Found: {len(orders)}\n")
    
    # Status mapping for better display
    status_display = {
        'draft': '📝 Draft',
        'sent': '📧 Quotation Sent',
        'sale': '✅ Sale Order',
        'done': '✔️ Locked',
        'cancel': '❌ Cancelled'
    }
    
    # Display orders
    for i, order in enumerate(orders, 1):
        print(f"[{i}] Order ID: {order['order_id']}")
        print(f"    Order Number: {order['order_number']}")
        print(f"    Customer ID: {order['customer_id']}")
        print(f"    Customer Number: {order['customer_number']}")
        print(f"    Customer Name: {order['customer_name']}")
        print(f"    Customer Phone: {order['customer_phone']}")
        print(f"    Status: {status_display.get(order['status'], order['status'].upper())}")
        print(f"    Salesperson: {order['salesperson']}")
        print(f"    Subtotal: ${order['untaxed_amount']:,.2f}")
        print(f"    Tax: ${order['tax_amount']:,.2f}")
        print(f"    Total: ${order['total_amount']:,.2f}")
        print(f"    Order Date: {order['order_date']}")
        print(f"    Created At: {order['created_at']}")
        print(f"    Company: {order['company']}")
        print("-" * 80)
    
    return orders


def export_to_json(orders):
    """Export orders to JSON file"""
    try:
        filename = f'sales_orders_{date.today()}.json'
        with open(filename, 'w') as f:
            json.dump(orders, f, indent=2, default=str)
        print(f"\n✓ Data exported to: {filename}")
    except Exception as e:
        print(f"❌ JSON export failed: {e}")


def export_to_csv(orders):
    """Export orders to CSV file"""
    try:
        import csv
        filename = f'sales_orders_{date.today()}.csv'
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            if orders:
                writer = csv.DictWriter(f, fieldnames=orders[0].keys())
                writer.writeheader()
                writer.writerows(orders)
        print(f"✓ Data exported to: {filename}")
    except Exception as e:
        print(f"❌ CSV export failed: {e}")


def main():
    """Main function to fetch and display sales orders from Odoo"""
    
    print("\n🔄 Connecting to Odoo...")
    
    # Fetch orders from Odoo
    orders = fetch_from_odoo()
    
    # Display orders
    display_orders(orders)
    
    # Export to files if orders found
    if orders:
        export_to_json(orders)
        export_to_csv(orders)
        print("\n✅ Process completed successfully!")
    else:
        print("\n⚠️ No data to export.")


if __name__ == "__main__":
    main()

In [ ]:
import xmlrpc.client
import ssl
import re
from html import unescape

# City filter (change as needed)
CITY_FILTER = 'Patna'  # Replace with desired city

def connect_odoo():
    """Establish connection to Odoo and authenticate"""
    try:
        # Create SSL context (for HTTPS connections)
        context = ssl._create_unverified_context()
        
        # Connect to authentication endpoint
        common = xmlrpc.client.ServerProxy(
            f'{ODOO_URL}/xmlrpc/2/common',
            context=context
        )
        
        # Authenticate and get user ID
        uid = common.authenticate(ODOO_DB, ODOO_USERNAME, ODOO_PASSWORD, {})
        
        if not uid:
            print("Authentication failed! Check credentials.")
            return None, None
        
        print(f"Successfully authenticated. User ID: {uid}")
        
        # Connect to object endpoint
        models = xmlrpc.client.ServerProxy(
            f'{ODOO_URL}/xmlrpc/2/object',
            context=context
        )
        
        return uid, models
    
    except Exception as e:
        print(f"Connection error: {e}")
        return None, None

def clean_html(html_text):
    """Remove HTML tags and convert to plain text"""
    if not html_text:
        return 'N/A'
    
    # Convert HTML entities to characters (e.g., &nbsp; to space)
    text = unescape(str(html_text))
    
    # Remove HTML tags
    text = re.sub(r'<[^>]+>', '', text)
    
    # Remove extra whitespace and newlines
    text = re.sub(r'\s+', ' ', text)
    
    # Strip leading/trailing whitespace
    text = text.strip()
    
    return text if text else 'N/A'

def fetch_customers_by_city(uid, models, city, sales_rep):
    """Fetch customers filtered by city"""
    try:
        # Define search domain (filter criteria)
        domain = [
            # ('customer_rank', '>', 0),  # Only customers (not suppliers)
            ('city', '=', city),  # City filter
            ('x_studio_sales_rep', '=', sales_rep)  # Sales representative filter (if needed)
        ]
        
        # Define fields to fetch
        fields = [
            'name',          # Customer name
            'street',        # Street address
            'phone',         # Phone number
            'x_studio_sales_rep',  # Custom field: Sales representative
            'comment'        # Notes/Internal notes
        ]
        
        # Search and read records
        customers = models.execute_kw(
            ODOO_DB, uid, ODOO_PASSWORD,
            'res.partner', 'search_read',
            [domain],
            {'fields': fields}
        )
        
        # Clean HTML from comment field
        for customer in customers:
            if 'comment' in customer:
                customer['comment'] = clean_html(customer['comment'])
        
        return customers
    
    except Exception as e:
        print(f"Error fetching customers: {e}")
        return []

def display_customers(customers):
    """Display customer information"""
    if not customers:
        print(f"No customers found with the specified filter.")
        return
    
    print(f"\n{'='*80}")
    print(f"Found {len(customers)} customer(s)")
    print(f"{'='*80}\n")
    
    for idx, customer in enumerate(customers, 1):
        print(f"Customer #{idx}")
        print(f"  ID: {customer.get('id', 'N/A')}")
        print(f"  Name: {customer.get('name', 'N/A')}")
        print(f"  Street: {customer.get('street', 'N/A')}")
        print(f"  Phone: {customer.get('phone', 'N/A')}")
        print(f"  Notes: {customer.get('comment', 'N/A')}")
        print(f"{'-'*80}\n")

def main():
    """Main execution function"""
    print("Connecting to Odoo...")
    
    # Connect and authenticate
    uid, models = connect_odoo()
    
    if not uid or not models:
        print("Failed to connect to Odoo. Exiting.")
        return
    
    # Fetch customers
    print(f"\nFetching customers from city: {CITY_FILTER}...")
    sales_rep = 'Manas'
    customers = fetch_customers_by_city(uid, models, CITY_FILTER, sales_rep)
    save_to_csv(customers, sales_rep)

    sales_rep = 'Satyam'
    customers = fetch_customers_by_city(uid, models, CITY_FILTER, sales_rep)
    save_to_csv(customers, sales_rep)

def save_to_csv(customers, name):
    """Save customer data to CSV file"""
    import csv
    from datetime import datetime
    
    if not customers:
        print("No data to export.")
        return
    
    filename = f"{name}_customers_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    
    try:
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=['id', 'name', 'street', 'phone', 'comment', 'x_studio_sales_rep'])
            writer.writeheader()
            writer.writerows(customers)
        
        print(f"Data exported successfully to {filename}")
    
    except Exception as e:
        print(f"Error exporting to CSV: {e}")

if __name__ == "__main__":
    main()